In [50]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [51]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.6)
result = llm.invoke("What is capital of India?")
print(result)

content='The capital of India is New Delhi.' response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 13, 'total_tokens': 21}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-d010311d-a34f-49b3-9a73-f0eb3715c695-0' usage_metadata={'input_tokens': 13, 'output_tokens': 8, 'total_tokens': 21}


In [52]:
llm1 = OpenAI(temperature=0.6)
result = llm1.invoke("What is capital of India?")
print(result)



The capital of India is New Delhi.


## Document Loading

In [53]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [54]:
from langchain_community.document_loaders import PyPDFLoader

In [55]:
loader = PyPDFLoader("D:\Data\LLM\llamaindex\IndianEconomy\Indian_Economy_Overview.pdf")
docs = loader.load()

In [56]:
len(docs)

9

In [60]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [57]:
## Split documents in to chunks

In [68]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(docs)

In [69]:
len(documents)

10

In [84]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('70e3768a-3577-4117-ab70-25956bb0d04f') or '70e3768a-3577-4117-ab70-25956bb0d04f'

# configure client
pc = Pinecone(api_key=api_key)

os.environ['PINECONE_API_KEY'] = '70e3768a-3577-4117-ab70-25956bb0d04f'

In [78]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [79]:
index_name = 'gen-qa-openai-fast'

In [80]:
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )
# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [81]:
embeddings = OpenAIEmbeddings()

query_result = embeddings.embed_query("Hello world")
len(query_result)

1536

In [82]:
from langchain_pinecone import PineconeVectorStore

In [85]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
        documents,
        index_name=index_name,
        embedding=embeddings
    )

In [86]:
retriever = vectorstore_from_docs.as_retriever()

In [87]:
prompt = hub.pull("rlm/rag-prompt")

In [88]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm1
    | StrOutputParser()
)

In [89]:
rag_chain.invoke("What are the Economic Reforms ?")

' The Economic Reforms refer to a series of policies and measures implemented in 1991 by the Indian government to liberalize and open up the economy. These reforms included deregulation of industries, reduction of import tariffs, liberalization of foreign direct investment policies, privatization of state-owned enterprises, and overhaul of the tax system. They aimed to spur economic growth, transform the economy into a more market-oriented one, and integrate it into the global economy.'

In [91]:
rag_chain.invoke("What are the Key Sectors of the Indian Economy?")

' Agriculture, Industry, and Services are the key sectors of the Indian economy. Agriculture employs around 42% of the workforce and contributes approximately 16% to the GDP, while the industrial sector contributes about 25% and the services sector accounts for around 55% of the GDP.'

In [92]:
rag_chain.invoke("What is capital of India?")

' The capital of India is New Delhi.'

In [93]:
rag_chain.invoke("What is capital of USA?")

" I don't know."

In [94]:
retriever = vectorstore_from_docs.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("What are the Key Sectors of the Indian Economy?")

len(retrieved_docs)

3

In [95]:
retrieved_docs

[Document(page_content="Indian Economy: An Overview\nKey Sectors of the Indian Economy\nAgriculture: Agriculture has historically been the backbone of the Indian economy. It employs around\n42% of the workforce and contributes approximately 16% to the GDP. Key agricultural products\ninclude rice, wheat, pulses, spices, and cotton. Despite its importance, the sector faces challenges\nsuch as low productivity, fragmented land holdings, and dependence on monsoons.\nIndustry: The industrial sector has witnessed significant growth post-reforms. It contributes about\n25% to the GDP and includes sub-sectors like manufacturing, mining, and construction. The 'Make\nin India' initiative launched in 2014 aims to boost manufacturing, attract FDI, and create jobs,\npositioning India as a global manufacturing hub.\nServices: The services sector is the largest contributor to the Indian economy, accounting for around\n55% of the GDP. It encompasses a wide range of activities including IT and IT-enable

In [96]:
rag_chain.invoke("What are the Key Sectors of the Indian Economy?")

' The key sectors of the Indian economy are agriculture, industry, and services. Agriculture is the largest employer, contributing around 16% to the GDP, while the industrial sector contributes approximately 25% and the services sector contributes around 55%.'